# New model class and loss using tf.keras

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from cbrain.imports import *

In [5]:
DATADIR = '/local/S.Rasp/preprocessed_data/'

In [7]:
!ls $DATADIR/000*

/local/S.Rasp/preprocessed_data//000_norm.nc
/local/S.Rasp/preprocessed_data//000_train.nc
/local/S.Rasp/preprocessed_data//000_train_shuffle.nc
/local/S.Rasp/preprocessed_data//000_valid.nc


## The training script

### Load the data generator

In [8]:
from cbrain.data_generator import DataGenerator

In [10]:
inputs = ['TBP', 'QBP', 'VBP', 'PS', 'SOLIN', 'SHFLX', 'LHFLX']
outputs = ['TPHYSTND', 'PHQ', 'FSNT', 'FSNS', 'FLNT', 'FLNS', 'PRECT']

In [12]:
from cbrain.normalization import conversion_dict

In [96]:
train_gen.data_ds.close(); valid_gen.data_ds.close()

In [97]:
train_gen = DataGenerator(
    DATADIR + '000_train.nc', inputs, outputs, DATADIR + '000_norm.nc', input_transform='max_rs', 
    output_transform=conversion_dict, shuffle=True
)

In [98]:
train_gen.input_transform.transform_arrays

{'mean': array([ 2.2894325e+02,  2.1785664e+02,  1.9873132e+02,  2.1808038e+02,
         2.1554680e+02,  2.0950041e+02,  2.0656183e+02,  2.0534174e+02,
         2.0475914e+02,  2.0452998e+02,  2.0499893e+02,  2.0714363e+02,
         2.0869292e+02,  2.1641628e+02,  2.2161829e+02,  2.2979195e+02,
         2.3853790e+02,  2.4752470e+02,  2.5425475e+02,  2.5984869e+02,
         2.6257587e+02,  2.6505743e+02,  2.6915845e+02,  2.7155261e+02,
         2.7324252e+02,  2.7473978e+02,  2.7640533e+02,  2.7802243e+02,
         2.8053253e+02,  2.8241229e+02,  1.7007696e-06,  1.6724131e-06,
         1.3829675e-06,  1.4767750e-06,  1.4249738e-06,  1.3743067e-06,
         1.3728851e-06,  1.4142012e-06,  1.5485415e-06,  2.0148816e-06,
         3.7566444e-06,  8.6058481e-06,  1.9792098e-05,  4.2598738e-05,
         8.8789493e-05,  1.7502054e-04,  3.2305956e-04,  5.7276123e-04,
         9.8182959e-04,  1.5745604e-03,  2.3750337e-03,  3.4777804e-03,
         4.7817174e-03,  5.5856109e-03,  6.1714784e-03, 

In [14]:
valid_gen = DataGenerator(
    DATADIR + '000_valid.nc', inputs, outputs, DATADIR + '000_norm.nc', input_transform='max_rs', 
    output_transform=conversion_dict, shuffle=False
)

### Build the model

In [15]:
from tensorflow.keras.layers import *

In [21]:
def act_layer(act):
    """Helper function to return regular and advanced activation layers"""
    return Activation(act) if act in tf.keras.activations.__dict__.keys() else tf.keras.layers.__dict__[act]()

In [103]:
Activation

tensorflow.python.keras.layers.core.Activation

In [47]:
def fc_model(input_shape, output_shape, hidden_layers, activation):
    
    inp = Input(shape=(input_shape,))
    
    # First hidden layer
    x = Dense(hidden_layers[0])(inp)
    x = act_layer(activation)(x)
    
    # Remaining hidden layers
    for h in hidden_layers[1:]:
        x = Dense(h)(x)
        x = act_layer(activation)(x)
    
    # Output layer
    out = Dense(output_shape)(x)
    
    return tf.keras.models.Model(inp, out)

In [48]:
model = fc_model(train_gen.n_inputs, train_gen.n_outputs, [128, 128], 'LeakyReLU')

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 94)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               12160     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 65)                8385      
Total params: 37,057
Trainable params: 37,057
Non-trainable params: 0
_________________________________________________________________


### Compile the model

In [104]:
model.compile(tf.keras.optimizers.Adam(0.1), loss='mse')

### Create learning rate scheduler

In [105]:
from tensorflow.keras.callbacks import LearningRateScheduler

In [78]:
class LRUpdate(object):
    def __init__(self, init_lr, step, divide):
        # From goo.gl/GXQaK6
        self.init_lr = init_lr
        self.step = step
        self.drop = 1./divide
        
    def __call__(self, epoch):
        lr = self.init_lr * np.power(self.drop, np.floor((epoch)/self.step))
        print(f'Learning rate = {lr}')
        return lr

In [79]:
lr_update = LRUpdate(0.001, 1, 5)

In [80]:
lr_update(2)

Learning rate = 4.000000000000001e-05


4.000000000000001e-05

### Train the model

In [106]:
model.fit_generator(train_gen, epochs=2, validation_data=valid_gen, callbacks=[LearningRateScheduler(lr_update)])

Learning rate = 0.001
Epoch 1/2
 294/3448 [=>............................] - ETA: 23s - loss: 0.0025

KeyboardInterrupt: 

### Save model file and text files

In [82]:
model.save('/local/S.Rasp/tmp/000_model.h5')

In [83]:
model.save_weights('/local/S.Rasp/tmp/000_model_weights.h5')

In [84]:
from cbrain.save_weights import save2txt

In [85]:
save2txt('/local/S.Rasp/tmp/000_model_weights.h5', '/local/S.Rasp/tmp/')

In [86]:
!ls /local/S.Rasp/tmp/

000_model.h5	      annos_ret_valid.csv  layer3_kernel.txt
000_model_weights.h5  clas_full.csv	   q2_1-32.png
1-imbalance.png       clas_prac.csv	   q2_2-32.png
1.pkl		      clas_ret.csv	   sample_SPCAM.nc
2-imbalance.png       layer1_bias.txt	   scatter_fqt_q2_1-32.png
2.pkl		      layer1_kernel.txt    scatter_fqt_q2_2-32.png
annos_full.csv	      layer2_bias.txt	   test.nc
annos_prac.csv	      layer2_kernel.txt    test_shuffle.nc
annos_ret_train.csv   layer3_bias.txt	   test.tfrecords


### Buuuut saving the normalization files will be a little more involved. Actually, it might be really easy

In [99]:
fmt = '%.6e'

In [100]:
def save_norm(input_transform, output_transform, save_dir):
    for name, arr in input_transform.transform_arrays.items():
        np.savetxt(save_dir + f'/inp_{name}.txt', arr.reshape(1, -1), fmt=fmt, delimiter=',')
    for name, arr in output_transform.transform_arrays.items():
        np.savetxt(save_dir + f'/out_{name}.txt', arr.reshape(1, -1), fmt=fmt, delimiter=',')


In [101]:
save_norm(train_gen.input_transform, train_gen.output_transform, '/local/S.Rasp/tmp/')

In [102]:
!ls /local/S.Rasp/tmp/

000_model.h5	      clas_full.csv	 layer3_kernel.txt
000_model_weights.h5  clas_prac.csv	 out_scale.txt
1-imbalance.png       clas_ret.csv	 q2_1-32.png
1.pkl		      inp_maxrs.txt	 q2_2-32.png
2-imbalance.png       inp_mean.txt	 sample_SPCAM.nc
2.pkl		      layer1_bias.txt	 scatter_fqt_q2_1-32.png
annos_full.csv	      layer1_kernel.txt  scatter_fqt_q2_2-32.png
annos_prac.csv	      layer2_bias.txt	 test.nc
annos_ret_train.csv   layer2_kernel.txt  test_shuffle.nc
annos_ret_valid.csv   layer3_bias.txt	 test.tfrecords


## some debugging for the shuffling script